In [1]:
import pandas as pd

In [2]:
df_gender = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')

In [3]:
set(df_gender.columns) & set(df_test.columns)

{'PassengerId'}

In [4]:
## Verificando se as keys de df gender são únicas
df_gender['PassengerId'].nunique() == len(df_gender)

True

In [5]:
## Verificando se as keys de df test são únicas
df_test['PassengerId'].nunique() == len(df_test)

True

In [6]:
## Agora que identifiquei que Passenger_ID é uma chave primaria posso juntar os dois DB's
df_merge = pd.merge(df_gender, df_test, on='PassengerId')

In [7]:
set(df_merge.columns)

{'Age',
 'Cabin',
 'Embarked',
 'Fare',
 'Name',
 'Parch',
 'PassengerId',
 'Pclass',
 'Sex',
 'SibSp',
 'Survived',
 'Ticket'}

In [8]:
df_merge

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [9]:
# notei que o df_merge na coluna cabin tem valores NA, vou verificar

df_merge['Cabin'].info()

## pelo info() é possivel verificar nonnul valores, verifiquei que de 418 entries apenas 91 são non-ni


<class 'pandas.core.series.Series'>
RangeIndex: 418 entries, 0 to 417
Series name: Cabin
Non-Null Count  Dtype 
--------------  ----- 
91 non-null     object
dtypes: object(1)
memory usage: 3.4+ KB


In [10]:
df_na = df_merge['Fare'].isna()

In [11]:
df_na

0      False
1      False
2      False
3      False
4      False
       ...  
413    False
414    False
415    False
416    False
417    False
Name: Fare, Length: 418, dtype: bool

In [12]:
df_merge.groupby(df_merge['Cabin'].isna())['Fare'].mean()

# A Média de 'Fare' de quem tem cabine demarcada para quem não tem é quase 5x maior, isso me indica que provávelmente minha hipotese esta certa NaN indica que não tem cabine demarcada e por isso pagou mais barato enquanto quem tinha, tinha sua cabine com número reservado.

Cabin
False    94.903162
True     19.080828
Name: Fare, dtype: float64

In [13]:
df_merge.groupby(df_merge['Cabin'].isna())['Fare'].count()

# Número de pessoas com Cabine demarcada ou não

Cabin
False     91
True     326
Name: Fare, dtype: int64

In [14]:
df_merge[df_merge['Fare'] == 0.0]

# Havia notado que tinha valores zerados em 'Fare', decidi verificar, são o presidente da companhia de criação do navio e o projetista.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
266,1158,0,1,"Chisholm, Mr. Roderick Robert Crispin",male,NaN,0,0,112051,0.0,NaN,S
372,1264,0,1,"Ismay, Mr. Joseph Bruce",male,49.0,0,0,112058,0.0,B52 B54 B56,S


In [15]:
#Agora vou analisar o PcClass, o nome me indica que é uma coluna que diz a classe do tripulante:

#1. Vou analisar se temos NaN ou valores zerados:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [16]:
#Em Pclass não temos NaN ou Null
# Agora vou agrupar por classe:
df_merge.groupby(df_merge['Pclass'])['Pclass'].count()

Pclass
1    107
2     93
3    218
Name: Pclass, dtype: int64

In [17]:
# O código anterior me diz que temos 200 pessoas de primeira e segunda classe contra 218 de terceira, já esperava que teriamos mais da terceira que da primeira e segunda
#No entanto o que me chama mais atenção é que temos mais da primeira classe do que dá segunda, seria por algo relacionado a "fare" paga?
df_merge.groupby(df_merge['Pclass'])['Fare'].mean()

Pclass
1    94.280297
2    22.202104
3    12.459678
Name: Fare, dtype: float64

In [18]:
## Acima da para verificar uma descrepância bem grande referente a segunda e terceira classe vs primeira.
# o que me chama atenção é a média da primeira classe ser parecida com o pessoal que tinha cabine separada, vamos investigar se quem tem cabine
# demarcada é obrigatóriamente primeira classe.
df_merge.groupby(df_merge['Pclass'])['Cabin'].count()

Pclass
1    80
2     7
3     4
Name: Cabin, dtype: int64